## This loading is pretty slow

In [22]:
import networkx as nx
import igraph as ig
import leidenalg

# G = np.loadtxt("carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv", delimiter=',')
graph = nx.read_weighted_edgelist("carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv", delimiter=",")

G = ig.Graph.from_networkx(graph)

## Largest

In [25]:
%%timeit

partition = leidenalg.find_partition(G, leidenalg.ModularityVertexPartition)

1min 5s ± 6.64 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Enron

In [28]:
# !tail -n +2 edgelist-enron-n184-forRB.csv > edgelist-enron-n184-forRB2.csv

In [30]:
enron = ig.Graph.from_networkx(nx.read_weighted_edgelist("edgelist-enron-n184-forRB2.csv", delimiter=","))

In [31]:
%%timeit

partition = leidenalg.find_partition(enron, leidenalg.ModularityVertexPartition)

1.83 ms ± 56.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## n500

In [32]:
n500 = ig.Graph.from_networkx(nx.read_weighted_edgelist("edgelist-n500-T40-forRB2.csv", delimiter=","))

In [33]:
%%timeit

partition = leidenalg.find_partition(n500, leidenalg.ModularityVertexPartition)

41.4 ms ± 375 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Networkit:ParallelLeiden

## MSR graph - 1.2GB

In [1]:
import networkit as nk

In [17]:
# Not sure why this library wants to know the ID of the first node. It's required arg
G = nk.graphio.readGraph("carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv", nk.Format.EdgeList, separator=',', firstNode=1)

print(G.numberOfNodes(), "nodes")
print(G.numberOfEdges(), "edges")

32277 nodes
26630547 edges


In [18]:
%%timeit
pl = nk.community.ParallelLeiden(G)

pl.run()

1.09 s ± 32.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
partition = pl.getPartition()

In [19]:
partition.subsetSizes()

[2266,
 382,
 1083,
 5755,
 3414,
 2796,
 335,
 1468,
 603,
 1102,
 1173,
 7281,
 785,
 45,
 3789]

## Enron graph

In [8]:
import numpy as np

G = np.loadtxt("carey-tsg-graphs/edgelist-enron-n184-forRB2.csv", delimiter=',', dtype=int)

In [9]:
print(np.min(G[:,0]))
print(np.max(G[:,1]))

0
183


In [19]:
# Firstnode ID has to be correct! 0 for enron
G = nk.graphio.readGraph("carey-tsg-graphs/edgelist-enron-n184-forRB2.csv", nk.Format.EdgeList, separator=',', firstNode=0)

print(G.numberOfNodes(), "nodes")
print(G.numberOfEdges(), "edges")

184 nodes
2215 edges


In [11]:
%%timeit
pl = nk.community.ParallelLeiden(G)

pl.run()

400 ms ± 23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## n500

In [14]:
import numpy as np

G = np.loadtxt("carey-tsg-graphs/edgelist-n500-T40-forRB2.csv", delimiter=',')#, dtype=int)

print(np.min(G[:,0])) # First Node ID
print(np.max(G[:,1]))

1.0
500.0


In [20]:
G = nk.graphio.readGraph("carey-tsg-graphs/edgelist-n500-T40-forRB2.csv", nk.Format.EdgeList, separator=',', firstNode=1)

print(G.numberOfNodes(), "nodes")
print(G.numberOfEdges(), "edges")

500 nodes
124548 edges


In [16]:
%%timeit
pl = nk.community.ParallelLeiden(G)

pl.run()

424 ms ± 20.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Graph Encoder Embedding

In [1]:
!ls ~/prog/GraphEmd/Main/Python

Benchmarks.ipynb   Encode.py	  Hyperperameters.py  __pycache__  tests
Clustering.py	   Evaluation.py  __init__.py	      Run.py	   utils
DataPreprocess.py  GNN.py	  LDA.py	      setup.py


In [2]:
!pwd

/users/ariel/prog/leiden-runtime-experiments


#### The below sometimes works and sometimes ModuleNotFound - why?

In [1]:
absolute_path = '/users/ariel/prog/'
graph_path = '/static/ariel'

# Change this depending on your folder structure

In [2]:
import sys
sys.path.append(absolute_path + 'GraphEmd/Main/Python')
import Evaluation # GEE's Evaluation.py

In [3]:
!ls /static/ariel/carey-tsg-graphs

edgelist-enron-n184-forRB2.csv	    edgelist-n500-T40-forRB2.csv
edgelist-msr-n32277-T24-forRB2.csv


## MSR 1.2GB

In [ ]:
G_edgelist, Y, n = Evaluation.loadGraph(graph_path + "/carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv", weighted=True, randomY=True)

Loading /static/ariel/carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv


In [ ]:
%%timeit
Z, W = Evaluation.graph_encoder_embed(G_edgelist, Y, n, Correlation = False, Laplacian = False)

## Enron

In [10]:
G_edgelist, Y, n = Evaluation.loadGraph(graph_path + "/carey-tsg-graphs/edgelist-enron-n184-forRB2.csv", weighted=True, randomY=True)

Loading /static/ariel/carey-tsg-graphs/edgelist-enron-n184-forRB2.csv


In [11]:
%%timeit
Z, W = Evaluation.graph_encoder_embed(G_edgelist, Y, n, Correlation = False, Laplacian = False)

3.24 ms ± 1.83 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
!ls /static/ariel/carey-tsg-graphs

edgelist-enron-n184-forRB2.csv	    edgelist-n500-T40-forRB2.csv
edgelist-msr-n32277-T24-forRB2.csv


## n500

In [6]:
G_edgelist, Y, n = Evaluation.loadGraph(graph_path + "/carey-tsg-graphs/edgelist-n500-T40-forRB2.csv", weighted=True, randomY=True)

Loading /static/ariel/carey-tsg-graphs/edgelist-n500-T40-forRB2.csv


In [8]:
%%timeit
Z, W = Evaluation.graph_encoder_embed(G_edgelist, Y, n, Correlation = False, Laplacian = False)

16 ms ± 4.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
